In [ ]:
from collections import Counter
from collections import defaultdict
import string
import numpy as np
import pandas as pd
from nltk import word_tokenize, ngrams
import time

import matplotlib.pyplot as plt


class NgramModel(object):

    def __init__(self, n=1):
        """
        Class init
        :param 'n': length of n-gram, default is one
        """
        self.n = n
        # a nested breakdown of all the words associated
        # before and after the appearance of another word
        self.entropylist = defaultdict(list)
        # keeps track of how many times ngram has appeared in the text before
        self.ngram_counter = {}

    def get_ngrams(self, corpus: list) -> dict:
        """
        Get ngrams
        :param 'corpus' : contents want to process in ngram
        :return         : list of ngrams
        """

        # list of special characters want to be removed from the corpus.
        removals = string.punctuation + '``'+'’'

        for com in corpus:
            ngram_statement = [str(i).lower() for i in ngrams(
                [iter for iter in word_tokenize(com) if iter not in removals], self.n)]
            counter = 0
            recent_list = []
            for gram in ngram_statement:

                # Romovig formatting that applied to the string format
                # when being passed through word_tokenize.
                if self.n == 1:
                    gram_clean = gram[2:len(gram)-3]
                else:
                    gram_clean = ''.join(gram)

                # Depending on the position and length of the gram
                if counter == 0:
                    self.entropylist['[start]'].append(gram_clean)
                    recent_list.append(gram_clean)
                elif counter > 0:
                    self.entropylist[str(
                        recent_list[len(recent_list)-self.n])].append(str(gram_clean))
                    recent_list.append(gram_clean)
                elif counter == len(ngram_statement):
                    self.entropylist[str(
                        recent_list[len(recent_list)-self.n])].append('[end]')
                    recent_list.append('[end]')

                counter += 1

        # usage count represents the appearance
        # of words (in their respective order)
        for key in self.entropylist:
            count_vals = {}
            # increment the appearance of the grams that appear within the gram.
            for val in self.entropylist[key]:
                if str(val) in count_vals:
                    count_vals[str(val)] += 1
                else:
                    count_vals[str(val)] = 1

            self.ngram_counter[str(val)] = [count_vals]

        return {'usage': self.ngram_counter, 'entropylist': self.entropylist}


class MarkovChainModel(object):

    def __init__(self, ngrams):
        """
        Class init
        :param 'ngrams': data structure of the n-gram
        """

        self.ngrams = ngrams

    def _transition_table(self) -> dict:
        """
        Returns the set of transition tables based on the length of n
        :return : set of transition table(s)
        """

        all_entropy = {}
        ngrams = self.ngrams

        for key in ngrams['entropylist']:
            cond_prob_val = {}
            relative_usage = Counter(ngrams['entropylist'][key])
            relative_words_len = sum(relative_usage.values())

            for following_gram in relative_usage:
                cond_prob_val[following_gram] = float(
                    relative_usage[following_gram]) / float(relative_words_len)

            all_entropy[key] = cond_prob_val

        return all_entropy

    def _melt_transition_table(self) -> pd.DataFrame:
        """
        Create a melted dataframe
        :return : dataframe of transition table(s)
        """

        transition_table = self._transition_table()
        flat_output = []
        for key in transition_table:
            for foll in transition_table[key]:
                temp_row = [key, foll, transition_table[key][foll]]
                flat_output.append(temp_row)
        pd_flat = pd.DataFrame(flat_output)
        headers = ['parent', 'relation', 'percentage']
        pd_flat.columns = headers
        # pd_flat.to_csv('output.csv')
        return pd_flat

    def get_matrix(self) -> pd.DataFrame:
        """
        Create a markov chain transition matrix
        :return : transition matrix of markov chain
        """
        transition_table = self._melt_transition_table()

        distinct = (list(set(transition_table['parent'].unique())
                    | set(transition_table['relation'].unique())))
        zero_data = np.zeros(shape=(len(distinct), len(distinct)))
        df = pd.DataFrame(index=distinct, columns=distinct, data=zero_data)
        for _, row in transition_table.iterrows():
            df[row['parent']][row['relation']] = row['percentage']
        return df


In [ ]:
def get_corpus(path: str) -> list:
    """
    Get corpus from the path file
    :param 'path'   : corpus file path
    :return         : generated corpus
    """
    corpus = ''
    with open(path, mode='r', encoding='utf8') as f:
        corpus = f.readlines()
        corpus = [s.rstrip('\n') for s in corpus]

    return corpus


In [ ]:
def strassen_algorithm(matrix, c_word, k):
    """
    Execute Markov chain to get the probability of the word come after current word
    :param 'matrix' : transition matrix of markov chain
    :param 'c_word' : current word
    :param 'k'      : the probability of k+1-th word comes after k-th word
    :return : probability of the next word
    """
    if k == 1:
        return matrix[c_word]
    else:
        res = pd.DataFrame(data=np.identity(len(matrix.index)),
                           index=matrix.index, columns=matrix.columns)
        for _ in range(k):
            res = res.dot(matrix)
        return res[c_word]


In [ ]:
from scipy.sparse import csr_matrix


def sparse_matrix_multiplication(matrix, c_word, k):
    mat = csr_matrix(matrix.to_numpy())
    if k == 1:
        return matrix[c_word]
    else:
        res = csr_matrix(np.identity(len(matrix.index)))
        for _ in range(k):
            res = res.dot(mat)
            # res = res@matrix
        res = pd.DataFrame(data=res.todense(),
                           index=matrix.index, columns=matrix.columns)
        return res[c_word]


In [ ]:
def mult(A, B):
    result = []  # final result
    for i in range(len(A)):
        row = []  # the new row in new matrix
        for j in range(len(B[0])):
            product = 0  # the new element in the new row
            for v in range(len(A[i])):
                product += B[i][v] * A[v][j]
            row.append(product)  # append sum of product into the new row
        result.append(row)  # append the new row into the final result
    return result


In [ ]:
def regular_matrix_multiplication(matrix, c_word, k):
    if k == 1:
        return matrix[c_word]
    else:
        A = list(matrix.to_numpy())
        res = list(np.identity(len(matrix.index)))
        for _ in range(k):
            res = mult(res, A)
        res = pd.DataFrame(data=np.array(
            res), index=matrix.index, columns=matrix.columns)
        return res[c_word]


In [ ]:
ngram_model = NgramModel()
path = r'Pride_and_Prejudice.txt'
corpus = get_corpus(path)

In [ ]:
n_len = [50,100,150,200,250,300,350,400,450,500]
strassen_rt = []
sparse_rt = []
regular_rt = []

for n in n_len:
    markov_model = MarkovChainModel(ngram_model.get_ngrams(corpus[:n]))
    matrix = markov_model.get_matrix()
    
    strassen_start = time.time()
    guess_word_1 = strassen_algorithm(matrix, 'universally', 3)
    strassen_rt.append(time.time() - strassen_start)
    # print(f'Language Model creating time with Strassen algorithm: {time.time() - start}')

    sparse_start = time.time()
    guess_word_2 = sparse_matrix_multiplication(matrix, 'universally', 3)
    sparse_rt.append(time.time() - sparse_start)
    # print(f'Language Model creating time with Sparse matrix algorithm: {time.time() - start}')

    regular_start = time.time()
    guess_word_2 = regular_matrix_multiplication(matrix, 'universally', 3)
    regular_rt.append(time.time() - regular_start)
    print(f'The count of words: {n}')
    # print(f'Language Model creating time with Sparse matrix algorithm: {time.time() - start}')


In [ ]:
print(strassen_rt)
print(sparse_rt)
print(regular_rt)

plt.grid()
plt.title("Markov Chain with different matrix multiplications")
plt.xlabel("The wordcount")
plt.ylabel("The runtime")

plt.plot(n_len, strassen_rt, 'g', label = 'Strassen matrix multiplication')
plt.plot(n_len, sparse_rt, 'r', label = 'Sparse matrix multiplication')
plt.plot(n_len, regular_rt, 'b', label = 'Regular matrix multiplication')
plt.legend()
plt.rcParams["figure.figsize"] = (20, 8)
plt.show()
plt.savefig('time-comparision-all.png')

In [ ]:
plt.plot(n_len, strassen_rt, 'g', label = 'Strassen matrix multiplication')
plt.plot(n_len, sparse_rt, 'r', label = 'Sparse matrix multiplication')
plt.rcParams["figure.figsize"] = (20, 8)
plt.legend()
plt.grid()
plt.title("Markov Chain with different matrix multiplications")
plt.xlabel("The wordcount")
plt.ylabel("The runtime")
plt.show()
plt.savefig('time-comparision-sparse-strassen.png')